In [2]:
import numpy as np
import random as r
from engine import Tensor

In [3]:
a = Tensor([[1.,2,3],[4,5,6]])
b = a.log()
b.backward()
print(a)
print(b)

Tensor:
([[1. 2. 3.]
 [4. 5. 6.]])
Grad:
([[1.         0.5        0.33333333]
 [0.25       0.2        0.16666667]])
Tensor:
([[0.         0.69314718 1.09861229]
 [1.38629436 1.60943791 1.79175947]])
Grad:
([[1. 1. 1.]
 [1. 1. 1.]])
